## Classificação por isolamento de frequência

Neste notebook será realizado um exemplo de classificação, isolando todas as frequências estimuladas no conjunto de dados `AVI SSVEP Single Target`. 

### Passos para a realização da classificação:

1. **Carrega** o arquivo `fif` (`mne.EpochsArray`) dos dados **filtrados**;
2. **Determine o limiar** para isolar cada uma das frequências estimuladas. Por exemplo, a faixa de frequência para o estímulo de 6.5 Hz irá resultar em pontos (`PSD`) que irão variar de 6.3 à 6.7 Hz, caso o limiar seja de 0.2 Hz;
3. **Obter a "energia"** do sinal por meio do cálculo `compute_psd` para cada uma das faixas de frequência que podem ser estimuladas. Por exemplo:
    - Obtenha todas as frequências estimuladas. Ex: 6, 6.5, 7, 7.5, 8.2 e 9.3;
    - Obtenha o valor mínimo e o máximo para cada frequência utilizando limiar. Ex: (5.8, 6.2), (6.3, 6.7), ...
    - Aplique o `compute_psd` para cada tupla (min, max), por meio dos parâmetros `fmin` e `fmax` do mesmo método.
4. Com as listas de pontos isoladas e computadas (`PSD`) para cada amostra, aplique um cálculo de característica adequada. Características manuais interessantes para este exemplo podem ser `max_value`, `average` ou `median`. No fim deste passo iremos obter um **vetor de características**;
5. Por fim, realize a **classificação**, que será um **cálculo de voto** simples (maior valor é provavelmente o a frequência evocada).

A seguir, um exemplo desta classificação com os dados `single target` de `AVI dataset`:

In [1]:
import mne

# tarefa 1
data = mne.read_epochs("../../datasets/avi/single/mne_data.fif")

C:\Users\vinny\AppData\Local\Temp\ipykernel_19900\2621150035.py:4: RuntimeWarning: This filename (../../datasets/avi/single/mne_data.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  data = mne.read_epochs("../../datasets/avi/single/mne_data.fif")


FileNotFoundError: File does not exist: "c:\Users\vinny\OneDrive\Documentos\UTFPR\Reconhecimento de Padroes\tutoriais\SSVEP\src\single\..\..\datasets\avi\single\mne_data.fif"

In [ ]:
# TAREFA 2
threshold = 0.25

# entendimento dos dados
print(data.get_data().shape)
data

(84, 1, 15360)


Number of events,84
Events,10: 126: 126.5: 127: 127.5: 128.2: 129.3: 12
Time range,0.000 – 29.998 s
Baseline,off


In [ ]:
# TAREFA 3

import numpy as np

targets = [float(item) for item in data.event_id.keys()]
print("Possíveis alvos:", targets)

features = list()
for i in range(len(data)):
    sample = list()
    for target in targets:
        fmin = target - threshold
        fmax = target + threshold
        sample.append(data[i].compute_psd(method='multitaper', fmin=fmin,
                                          fmax=fmax, verbose=False, n_jobs=8))
    features.append(sample)

X = np.array(features)
print("Formato dos dados calculados:", X.shape)

Possíveis alvos: [6.0, 6.5, 7.0, 7.5, 8.2, 9.3, 10.0]
Formato dos dados calculados: (84, 7, 1, 1, 15)


In [ ]:
# organizando os dados

X = X.reshape(X.shape[0], X.shape[1], X.shape[-1])
print("Formato padronizado dos dados com PSD calculado:", X.shape)

Formato padronizado dos dados com PSD calculado: (84, 7, 15)


In [ ]:
# TAREFA 4

# aplicando a característica de "maior valor"
max_values = np.max(X, axis=-1)
max_values.shape

(84, 7)

In [ ]:
# TAREFA 5

y = np.load("datasets/avi/single/labels_formated.npy")

# entendimento dos dados
print(y, y.shape, targets)

# classificação pelo "voto" do maior valor
i_max = max_values.argmax(axis=-1)
hits = [1 for i in range(len(i_max)) if targets[i_max[i]] == y[i]]
acc = 100 * sum(hits) / len(y)
print(f'\nPorcentagem de acerto: {acc:.2f}%')

[10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3
 10.  10.  10.   6.5  6.5  6.5  6.   6.   6.   7.5  7.5  7.5  7.   7.
  7.   8.2  8.2  8.2  9.3  9.3  9.3 10.  10.  10.   6.5  6.5  6.5  6.
  6.   6.   7.5  7.5  7.5  7.   7.   7.   8.2  8.2  8.2  9.3  9.3  9.3] (84,) [6.0, 6.5, 7.0, 7.5, 8.2, 9.3, 10.0]

Porcentagem de acerto: 85.71%
